In [126]:
import pandas as pd
import sklearn as skl
import xgboost as xgb
import category_encoders as ce
from datetime import timedelta

cols = ['created','ref_hash','implicit','device_brand','device_model','wifi','device_language','ref_type']
installs = pd.read_csv('/home/fedoremelianenko/FIUBA/Organizacion de datos/TPs/data/installs.csv',usecols=cols)
installs['created'] = pd.to_datetime(installs['created'])

def generar_ventanas(df, str_columna_fecha):
    FECHA_INICIO = pd.to_datetime(df[str_columna_fecha].min().strftime('%Y-%m-%d'))
    FECHA_FIN = pd.to_datetime(df[str_columna_fecha].max().strftime('%Y-%m-%d'))

    FECHA_UN_DIA = FECHA_INICIO + timedelta(days=1)
    FECHA_DOS_DIAS = FECHA_INICIO + timedelta(days=2)
    FECHA_TRES_DIAS = FECHA_INICIO + timedelta(days=3)
    FECHA_CUATRO_DIAS = FECHA_INICIO + timedelta(days=4)
    FECHA_CINCO_DIAS = FECHA_INICIO + timedelta(days=5)
    FECHA_SEIS_DIAS = FECHA_INICIO + timedelta(days=6)
    FECHA_SIETE_DIAS = FECHA_INICIO + timedelta(days=7)

    ventana_uno = installs.loc[(df[str_columna_fecha] < FECHA_TRES_DIAS)]
    ventana_uno = ventana_uno.loc[(df[str_columna_fecha] > FECHA_INICIO)]

    ventana_dos = installs.loc[(df[str_columna_fecha] < FECHA_CUATRO_DIAS)]
    ventana_dos = ventana_dos.loc[(df[str_columna_fecha] > FECHA_UN_DIA)]

    ventana_tres = installs.loc[(df[str_columna_fecha] < FECHA_CINCO_DIAS)]
    ventana_tres = ventana_tres.loc[(df[str_columna_fecha] > FECHA_DOS_DIAS)]
    
    ventana_cuatro = installs.loc[(df[str_columna_fecha] < FECHA_SEIS_DIAS)]
    ventana_cuatro = ventana_tres.loc[(df[str_columna_fecha] > FECHA_TRES_DIAS)]
    
    ventana_cinco = installs.loc[(df[str_columna_fecha] < FECHA_SIETE_DIAS)]
    ventana_cinco = ventana_tres.loc[(df[str_columna_fecha] > FECHA_CUATRO_DIAS)]
    
    return ventana_uno, ventana_dos, ventana_tres, ventana_cuatro, ventana_cinco
    
    

In [127]:
installs['implicit'] = installs['implicit'].apply(lambda x: 1 if x== True else 0)
installs['wifi'] = installs['wifi'].apply(lambda x: 1 if x== True else 0)
installs['ref_type'] = installs['ref_type'].apply(lambda x: 1 if (x == 1891515180541284343) else 0)
installs['device_language']=installs['device_language'].fillna(value=0)
installs.head()

,created,ref_type,ref_hash,implicit,device_brand,device_model,wifi,device_language
0,2019-03-13 01:43:33.445,1,8464844987297247076,0,3.083059e+17,4.445252e+18,0,0.000000e+00
1,2019-03-13 04:58:35.078,1,3250564871270161533,0,3.083059e+17,4.445252e+18,0,0.000000e+00
2,2019-03-13 04:20:57.666,1,7953318831018100268,1,5.137992e+17,6.026543e+18,0,3.301378e+18
3,2019-03-13 04:20:57.698,1,7953318831018100268,0,5.137992e+17,6.026543e+18,0,0.000000e+00
4,2019-03-10 22:24:56.684,1,8355110941557237501,1,1.083369e+18,7.876612e+18,0,3.301378e+18


In [128]:
encoder = ce.BinaryEncoder(cols=['device_model'])
installs = encoder.fit_transform(installs)
installs['cantidad_instalaciones'] = 1
installs['created'] = sorted(installs['created'])
installs.head()

,created,ref_type,ref_hash,implicit,device_brand,device_model_0,device_model_1,device_model_2,device_model_3,device_model_4,device_model_5,device_model_6,device_model_7,device_model_8,device_model_9,wifi,device_language,cantidad_instalaciones
0,2019-03-05 00:00:38.219,1,8464844987297247076,0,3.083059e+17,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,1
1,2019-03-05 00:05:35.234,1,3250564871270161533,0,3.083059e+17,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,1
2,2019-03-05 00:07:25.431,1,7953318831018100268,1,5.137992e+17,0,0,0,0,0,0,0,0,1,0,0,3.301378e+18,1
3,2019-03-05 00:07:38.572,1,7953318831018100268,0,5.137992e+17,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,1
4,2019-03-05 00:10:36.708,1,8355110941557237501,1,1.083369e+18,0,0,0,0,0,0,0,0,1,1,0,3.301378e+18,1


In [129]:
installs_ventana_uno, installs_ventana_dos, installs_ventana_tres, installs_ventana_cuatro, installs_ventana_cinco = generar_ventanas(installs,'created')

In [130]:
iv1_grouped = installs_ventana_uno.groupby('ref_hash').agg({'cantidad_instalaciones':'sum'})
installs_ventana_uno = installs_ventana_uno.merge(iv1_grouped, on='ref_hash',how='left')
installs_ventana_uno.drop(['cantidad_instalaciones_x'], axis=1, inplace=True)
installs_ventana_uno.rename({'cantidad_instalaciones_y':'cantidad_instalaciones'}, inplace=True)
installs_ventana_uno.head()

,created,ref_type,ref_hash,implicit,device_brand,device_model_0,device_model_1,device_model_2,device_model_3,device_model_4,device_model_5,device_model_6,device_model_7,device_model_8,device_model_9,wifi,device_language,cantidad_instalaciones_y
0,2019-03-05 00:00:38.219,1,8464844987297247076,0,3.083059e+17,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,1
1,2019-03-05 00:05:35.234,1,3250564871270161533,0,3.083059e+17,0,0,0,0,0,0,0,0,0,1,0,0.000000e+00,1
2,2019-03-05 00:07:25.431,1,7953318831018100268,1,5.137992e+17,0,0,0,0,0,0,0,0,1,0,0,3.301378e+18,2
3,2019-03-05 00:07:38.572,1,7953318831018100268,0,5.137992e+17,0,0,0,0,0,0,0,0,1,0,0,0.000000e+00,2
4,2019-03-05 00:10:36.708,1,8355110941557237501,1,1.083369e+18,0,0,0,0,0,0,0,0,1,1,0,3.301378e+18,2
